# Time Series Analysis of NAICS

<img src="img.svg.png">

The North American Industry Classification System or NAICS is a classification of business establishments by type of economic activity. 
It is used by government and business in Canada, Mexico, and the United States of America.

It has largely replaced the older Standard Industrial Classification (SIC) system, except in some government agencies, such as the U.S. Securities and Exchange Commission (SEC).

An establishment is typically a single physical location, though administratively distinct operations at a single location may be treated as distinct establishments. 
Each establishment is classified to an industry according to the primary business activity taking place there. NAICS does not offer guidance on the classification of enterprises (companies) which are composed of multiple establishments.

NAICS is designed to provide common definitions of the
industrial structure of the three countries and a common statistical framework to facilitate the
analysis of the three economies.

## The data provided contains:

<b>a- Raw data:</b><br>
15 CSV files beginning with RTRA.<br>
These files contain employment data by industry at different levels of aggregation; 2-digit NAICS, 3-digit NAICS, and 4-digit
NAICS. <br>
<ul>
Columns mean as follows:
    <li>
 SYEAR: Survey Year
        </li>
    <li>
 SMTH: Survey Month
        </li>
    <li>
 NAICS: Industry name and associated NAICS code in the bracket
        </li>
    <li>
 _EMPLOYMENT_: Employment
        </li>
</ul>

<b>b- LMO Detailed Industries by NAICS:</b><br> 
An excel file for mapping the RTRA data to the desired data. <br>
The first column of this file has a list of 59 industries that are frequently used.<br>
The second column has their NAICS definitions. <br>
Using these NAICS definitions and RTRA data, you would create a monthly employment data series from 1997 to 2018 for these 59
industries.


<b>c- Data Output Template:</b><br>
An excel file with an empty column for employment. 

## Task

In this task, we need to understand how the NAICS works as a hierarchical structure for defining industries at different levels of aggregation. <br>

<b>For example:</b> <br>
In NAICS 2017 – Statistics Canada.pdf (see page 22), a 2-digit NAICS industry (e.g., 23 - Construction) is
composed of some 3-digit NAICS industries (236 - Construction of buildings, 237 - Heavy
and civil engineering construction, and a few more 3-digit NAICS industries).<br>

Similarly, a 3-digit NAICS industry (e.g., 236 - Construction of buildings), is composed of
4-digit NAICS industries (2361 - Residential building construction and 2362 -Non-residential building construction).

##  Get, and prepare the Dataset:

### a- Loading and exploring the LMO_Detailed_Industries_by_NAICS data:

In [34]:
# import libraries
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as snb
import matplotlib.pyplot as plt

In [75]:
# Loading LMO_Detailed_Industries_by_NAICS data
LMO_Detailed_Industries_df = pd.read_excel("Data\LMO_Detailed_Industries_by_NAICS.xlsx")
LMO_Detailed_Industries_df.head()

,LMO_Detailed_Industry,NAICS
0,Farms,111 & 112
1,"Fishing, hunting and trapping",114
2,Forestry and logging,113
3,Support activities for agriculture and forestry,115
4,Oil and gas extraction,211


In [76]:
# Create a list of NAICS for industries
LMO_Detailed_Industries_df['NAICS'] = LMO_Detailed_Industries_df['NAICS'].replace({'&':','}, regex=True)

In [77]:
LMO_Detailed_Industries_df.head()

,LMO_Detailed_Industry,NAICS
0,Farms,"111 , 112"
1,"Fishing, hunting and trapping",114
2,Forestry and logging,113
3,Support activities for agriculture and forestry,115
4,Oil and gas extraction,211


In [78]:
LMO_Detailed_Industries_df=LMO_Detailed_Industries_df[['NAICS','LMO_Detailed_Industry']]

LMO_Detailed_Industries_df1=LMO_Detailed_Industries_df[~LMO_Detailed_Industries_df['NAICS'].str.contains(',', na=False)]
LMO_Detailed_Industries_df2=LMO_Detailed_Industries_df[LMO_Detailed_Industries_df['NAICS'].str.contains(',', na=False)]


LMO_Detailed_Industries_df2=LMO_Detailed_Industries_df2.assign(NAICS=LMO_Detailed_Industries_df2['NAICS'].str.split(',')).explode('NAICS')


In [79]:
LMO_Detailed_Industries_df=LMO_Detailed_Industries_df1.append(LMO_Detailed_Industries_df2, ignore_index=True)
LMO_Detailed_Industries_df.columns=['Code','LMO_Detailed_Industry']

In [80]:
LMO_Detailed_Industries_df

,Code,LMO_Detailed_Industry
0,114,"Fishing, hunting and trapping"
1,113,Forestry and logging
2,115,Support activities for agriculture and forestry
3,211,Oil and gas extraction
4,213,Support activities for mining and oil and gas ...
...,...,...
102,518,"Broadcasting, data processing, and information"
103,519,"Broadcasting, data processing, and information"
104,913,Local and Indigenous public administration
105,914,Local and Indigenous public administration


### b- Loading and exploring the 2-Digit NAICS Industries data:

In [41]:
# All the csv files have the same columns

In [42]:
# Get the data of 2digit NAICS industries
df_2_NAICS = pd.concat(map(pd.read_csv, ['Data/RTRA_Employ_2NAICS_00_05.csv', 'Data/RTRA_Employ_2NAICS_06_10.csv',
                                         'Data/RTRA_Employ_2NAICS_11_15.csv', 'Data/RTRA_Employ_2NAICS_16_20.csv',
                                         'Data/RTRA_Employ_2NAICS_97_99.csv']))

In [43]:
df_2_NAICS.head()

,SYEAR,SMTH,NAICS,_EMPLOYMENT_
0,2000,1,Accommodation and food services [72],148000
1,2000,1,"Administrative and support, waste management a...",59250
2,2000,1,"Agriculture, forestry, fishing and hunting [11]",61750
3,2000,1,"Arts, entertainment and recreation [71]",39500
4,2000,1,Construction [23],106250


In [44]:
df_2_NAICS.tail()

,SYEAR,SMTH,NAICS,_EMPLOYMENT_
715,1999,12,"Real estate, rental and leasing [53]",37000
716,1999,12,Retail trade [44-45],230750
717,1999,12,Transportation and warehousing [48-49],117500
718,1999,12,Utilities [22],10250
719,1999,12,Wholesale trade [41],74000


In [45]:
df_2_NAICS.shape

(5472, 4)

### Separate the Industry description and NAICS code 

In [50]:
# Separate the Industry description and NAICS code 
code2 = pd.DataFrame(df_2_NAICS['NAICS'].str.split('[').tolist(), columns=["NAICS","Code"])
code2 = pd.DataFrame(code2['Code'].str.replace(']', '').str.split('-').tolist(), columns=["Code", "upper_code"])

In [51]:
code2

,Code,upper_code
0,72,None
1,56,None
2,11,None
3,71,None
4,23,None
...,...,...
5467,53,None
5468,44,45
5469,48,49
5470,22,None


In [52]:
df_2_NAICS["Code"] = code2["Code"]
df_2_NAICS["upper_code"] = code2["upper_code"]

In [53]:
df_2_NAICS

,SYEAR,SMTH,NAICS,_EMPLOYMENT_,Code,upper_code
0,2000,1,Accommodation and food services [72],148000,72,None
1,2000,1,"Administrative and support, waste management a...",59250,56,None
2,2000,1,"Agriculture, forestry, fishing and hunting [11]",61750,11,None
3,2000,1,"Arts, entertainment and recreation [71]",39500,71,None
4,2000,1,Construction [23],106250,23,None
...,...,...,...,...,...,...
715,1999,12,"Real estate, rental and leasing [53]",37000,53,None
716,1999,12,Retail trade [44-45],230750,44,45
717,1999,12,Transportation and warehousing [48-49],117500,48,49
718,1999,12,Utilities [22],10250,22,None


### c- Loading and exploring the 3-Digit NAICS Industries data:

In [54]:
# Get the data of 3digit NAICS industries
df_3_NAICS = pd.concat(map(pd.read_csv, ['Data/RTRA_Employ_3NAICS_00_05.csv','Data/RTRA_Employ_3NAICS_06_10.csv', 
                                         'Data/RTRA_Employ_3NAICS_11_15.csv','Data/RTRA_Employ_3NAICS_16_20.csv',
                                         'Data/RTRA_Employ_3NAICS_97_99.csv']))

In [55]:
df_3_NAICS

,SYEAR,SMTH,NAICS,_EMPLOYMENT_
0,2000,1,Aboriginal public administration[914],500
1,2000,1,Accommodation services[721],33750
2,2000,1,Administrative and support services[561],55250
3,2000,1,Air transportation[481],17500
4,2000,1,Ambulatory health care services[621],53000
...,...,...,...,...
3703,1999,12,Utilities[221],10000
3704,1999,12,Warehousing and storage[493],4500
3705,1999,12,Waste management and remediation services[562],4500
3706,1999,12,Water transportation[483],6750


In [56]:
# Separate the Industry description and NAICS code 
code3 = pd.DataFrame(df_3_NAICS['NAICS'].str.split('[').tolist(), columns=["NAICS","code"])
code3 = pd.DataFrame(code3['code'].str.replace(']', ''))

In [57]:
df_3_NAICS["Code"] = code3["code"]
df_3_NAICS["upper_code"] = None

In [58]:
df_3_NAICS

,SYEAR,SMTH,NAICS,_EMPLOYMENT_,Code,upper_code
0,2000,1,Aboriginal public administration[914],500,914,None
1,2000,1,Accommodation services[721],33750,721,None
2,2000,1,Administrative and support services[561],55250,561,None
3,2000,1,Air transportation[481],17500,481,None
4,2000,1,Ambulatory health care services[621],53000,621,None
...,...,...,...,...,...,...
3703,1999,12,Utilities[221],10000,413,None
3704,1999,12,Warehousing and storage[493],4500,113,None
3705,1999,12,Waste management and remediation services[562],4500,526,None
3706,1999,12,Water transportation[483],6750,442,None


### d- Loading and exploring the 4-Digit NAICS Industries data:

In [59]:
# Get the data of 3digit NAICS industries
df_4_NAICS = pd.concat(map(pd.read_csv, ['Data/RTRA_Employ_4NAICS_00_05.csv','Data/RTRA_Employ_4NAICS_06_10.csv', 
                                         'Data/RTRA_Employ_4NAICS_11_15.csv','Data/RTRA_Employ_4NAICS_16_20.csv',
                                         'Data/RTRA_Employ_4NAICS_97_99.csv']))

In [60]:
df_4_NAICS.head()

,SYEAR,SMTH,NAICS,_EMPLOYMENT_
0,2000,1,1100,500
1,2000,1,1111,0
2,2000,1,1112,2000
3,2000,1,1113,250
4,2000,1,1114,7750


In [61]:
# Separate the Industry description and NAICS code 
df_4_NAICS['Code'] = df_4_NAICS['NAICS'] 
df_4_NAICS["upper_code"] = None

In [62]:
df_4_NAICS.head()

,SYEAR,SMTH,NAICS,_EMPLOYMENT_,Code,upper_code
0,2000,1,1100,500,1100,None
1,2000,1,1111,0,1111,None
2,2000,1,1112,2000,1112,None
3,2000,1,1113,250,1113,None
4,2000,1,1114,7750,1114,None


In [63]:
df_4_NAICS.shape

(85572, 6)

In [64]:
df_4_NAICS.tail()

,SYEAR,SMTH,NAICS,_EMPLOYMENT_,Code,upper_code
11119,1999,12,9111,2250,9111,None
11120,1999,12,9120,28500,9120,None
11121,1999,12,9130,30250,9130,None
11122,1999,12,9141,500,9141,None
11123,1999,12,9191,750,9191,None


In [65]:
df_4_NAICS.isna().sum()

SYEAR               0
SMTH                0
NAICS               0
_EMPLOYMENT_        0
Code                0
upper_code      85572
dtype: int64

4 digits NAICS dosenot have LMO_Detailed_Industry.

## Combine all the 2, 3 and 4 digits NAICS

In [66]:
combined_df = (df_2_NAICS.append(df_3_NAICS)).append(df_4_NAICS) 

In [67]:
combined_df

,SYEAR,SMTH,NAICS,_EMPLOYMENT_,Code,upper_code
0,2000,1,Accommodation and food services [72],148000,72,None
1,2000,1,"Administrative and support, waste management a...",59250,56,None
2,2000,1,"Agriculture, forestry, fishing and hunting [11]",61750,11,None
3,2000,1,"Arts, entertainment and recreation [71]",39500,71,None
4,2000,1,Construction [23],106250,23,None
...,...,...,...,...,...,...
11119,1999,12,9111,2250,9111,None
11120,1999,12,9120,28500,9120,None
11121,1999,12,9130,30250,9130,None
11122,1999,12,9141,500,9141,None


## Joining codes to the output template, To get the data_output formated result


In [85]:
#Read 'Data_Output_Template' file

output_df = pd.read_excel('Data\Data_Output_Template.xlsx')

In [87]:
#join tables
output_df= output_df.merge(LMO_Detailed_Industries_df,how='left', on='LMO_Detailed_Industry')

In [88]:
output_df.head()

,SYEAR,SMTH,LMO_Detailed_Industry,Employment,Code
0,1997,1,Accommodation services,NaN,721
1,1997,1,Air transportation,NaN,481
2,1997,1,Ambulatory health care services,NaN,621
3,1997,1,"Amusement, gambling and recreation industries",NaN,713
4,1997,1,"Architectural, engineering and related services",NaN,5413


In [90]:
cols = ['SYEAR', 'SMTH', 'Code']
output_df['check']= output_df[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)


In [93]:
output_df.head()

,SYEAR,SMTH,LMO_Detailed_Industry,Employment,Code,check
0,1997,1,Accommodation services,NaN,721,1997_1_721
1,1997,1,Air transportation,NaN,481,1997_1_481
2,1997,1,Ambulatory health care services,NaN,621,1997_1_621
3,1997,1,"Amusement, gambling and recreation industries",NaN,713,1997_1_713
4,1997,1,"Architectural, engineering and related services",NaN,5413,1997_1_5413


In [91]:
combined_df['check']= combined_df[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

combined_df=combined_df[['check', '_EMPLOYMENT_']]

In [94]:
combined_df.head()

,check,_EMPLOYMENT_
0,2000_1_72,148000
1,2000_1_56,59250
2,2000_1_11,61750
3,2000_1_71,39500
4,2000_1_23,106250


In [95]:
output_df=output_df.merge(combined_df,how='left', on='check')


In [96]:
output_df

,SYEAR,SMTH,LMO_Detailed_Industry,Employment,Code,check,_EMPLOYMENT_
0,1997,1,Accommodation services,NaN,721,1997_1_721,24000.0
1,1997,1,Air transportation,NaN,481,1997_1_481,17000.0
2,1997,1,Ambulatory health care services,NaN,621,1997_1_621,40000.0
3,1997,1,"Amusement, gambling and recreation industries",NaN,713,1997_1_713,20000.0
4,1997,1,"Architectural, engineering and related services",NaN,5413,1997_1_5413,27000.0
...,...,...,...,...,...,...,...
28258,2018,12,Utilities,NaN,22,2018_12_22,106500.0
28259,2018,12,Warehousing and storage,NaN,493,2018_12_493,19250.0
28260,2018,12,Water transportation,NaN,483,2018_12_483,0.0
28261,2018,12,Wholesale trade,NaN,41,2018_12_41,51750.0


In [97]:
output_df.drop(['Employment','check'],axis=1,inplace=True)
output_df.columns=['SYEAR', 'SMTH', 'LMO_Detailed_Industry', 'CODE', 'Employment']

In [98]:
output_df

,SYEAR,SMTH,LMO_Detailed_Industry,CODE,Employment
0,1997,1,Accommodation services,721,24000.0
1,1997,1,Air transportation,481,17000.0
2,1997,1,Ambulatory health care services,621,40000.0
3,1997,1,"Amusement, gambling and recreation industries",713,20000.0
4,1997,1,"Architectural, engineering and related services",5413,27000.0
...,...,...,...,...,...
28258,2018,12,Utilities,22,106500.0
28259,2018,12,Warehousing and storage,493,19250.0
28260,2018,12,Water transportation,483,0.0
28261,2018,12,Wholesale trade,41,51750.0
